In [1]:
import pathlib
import pandas as pd
import numpy as np

## Configuration and Inputs

First let's set up the environment.

Let's start with a few inputs, defined below.

Input: pipeline inputs (raw data) directory:

In [2]:
RAW_DATA_DIR = pathlib.Path('../../data/raw')

List of ENCODE files (from pipeline's raw data)

In [3]:
ENCODE_METADATA_TSV = RAW_DATA_DIR / 'encode/encode_metadata.2021-11-05.tsv.gz'
assert ENCODE_METADATA_TSV.is_file()

**Input**: blacklist file for hg38

In [4]:
BLACKLIST = RAW_DATA_DIR / 'blacklist/hg38-blacklist.v2.bed.gz'

Input: chromsizes file for `hg38`, only the canonical chromosomes

In [5]:
CHROMSIZES = RAW_DATA_DIR / 'genome/hg38.filtered.chrom.sizes'

Get a set of valid chromosomes from chromsizes file

In [6]:
VALID_CHROMOSOMES = set()

with open(CHROMSIZES, 'rt') as f:
    for line in f:
        VALID_CHROMOSOMES.add(line.partition('\t')[0])
VALID_CHROMOSOMES

{'chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr20',
 'chr21',
 'chr22',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrX'}

**Config**: the configuration directives required for computations, as they're defined in the [main config file](../../config/config.yaml).

In [7]:
BIN_SIZE = 1000  
STATISTICS_PSEUDOCOUNT = 100
# We only need the number below for the verification step (to load the correct results file)
MIN_PERIODS_FOR_CORRELATIONS = 1

While the pipeline analyses the ENCODE data exhaustively, in this script we will only process the data for one `BED` dataset that we will call $X$. We define this dataset by its ENCODE identifier below.

We will becomparing this dataset $X$ to a chromatin state defined by it's shorthand name.
You can find the list of available chromatin states and their shorthands [on the Roadmap website](https://egg2.wustl.edu/roadmap/web_portal/chr_state_learning.html).

In [8]:
ENCODE_IDENTIFIER_BEDFILE_X = 'ENCFF981ISM' # PHF8
CHROMATIN_STATE = '1_TssA'

We will write outputs to the directory specified by the combination of these identifeirs:

In [9]:
NOTEBOOK_OUTPUT_DIR = pathlib.Path('outputs') / f'{ENCODE_IDENTIFIER_BEDFILE_X}-{CHROMATIN_STATE}'
if not NOTEBOOK_OUTPUT_DIR.is_dir():
    NOTEBOOK_OUTPUT_DIR.mkdir(parents=True)

## Downloading ENCODE data

Just like the pipeline, we can extract the information about ENCODE files from the encode metadata TSV:

In [10]:
encode_meta = pd.read_csv(ENCODE_METADATA_TSV, sep='\t', index_col=0)
encode_meta = encode_meta.loc[[ENCODE_IDENTIFIER_BEDFILE_X]]
encode_meta

<ipython-input-10-7f394826646b>:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  encode_meta = pd.read_csv(ENCODE_METADATA_TSV, sep='\t', index_col=0)


,File format,File type,File format type,Output type,File assembly,Experiment accession,Assay,Donor(s),Biosample term id,Biosample term name,...,Genome annotation,Platform,Controlled by,File Status,s3_uri,File analysis title,File analysis status,Audit WARNING,Audit NOT_COMPLIANT,Audit ERROR
File accession,,,,,,,,,,,,,,,,,,,,,
ENCFF981ISM,bed narrowPeak,bed,narrowPeak,IDR thresholded peaks,GRCh38,ENCSR000AQH,TF ChIP-seq,/human-donors/ENCDO000AAD/,EFO:0002067,K562,...,NaN,NaN,NaN,released,s3://encode-public/2021/01/07/4e27a806-b056-4a...,ENCODE4 v1.6.1 GRCh38,released,"borderline replicate concordance, low read len...",NaN,NaN


The file can be downloaded from the columns `File download URL` and `md5sum`

In [11]:
encode_meta[['File download URL', 'md5sum']]

,File download URL,md5sum
File accession,,
ENCFF981ISM,https://www.encodeproject.org/files/ENCFF981IS...,9c193f16fb52a60daa00555fc986a6a1


For simplicity we will keep using the file identifier as filename, and will be downloading the file to the following location:

In [12]:
BED_FILENAMES = {
    id_: (NOTEBOOK_OUTPUT_DIR / f'{id_}.bed.gz') for id_ in encode_meta.index
}
BED_FILENAMES

{'ENCFF981ISM': PosixPath('outputs/ENCFF981ISM-1_TssA/ENCFF981ISM.bed.gz')}

In [13]:
import urllib.request
import hashlib

for id_, row in encode_meta[['File download URL', 'md5sum']].iterrows():
    url = row['File download URL']
    expected_checksum = row['md5sum']
    target_filename = BED_FILENAMES[id_]
    
    print(f"-> Downloading {url} to {target_filename}")
    urllib.request.urlretrieve(url, target_filename)
    
    print(f"-> Verifying checksum")
    
    # Computes MD5 checksum for file, see https://stackoverflow.com/a/59056796/171400
    with open(target_filename, 'rb') as f:
        checksum = hashlib.md5()
        while chunk := f.read(8192):
            checksum.update(chunk)
        
        checksum = checksum.hexdigest()
    print(f"-> Downloaded file checksum: {checksum}")
    assert checksum == expected_checksum, "Checksums do not match"

-> Downloading https://www.encodeproject.org/files/ENCFF981ISM/@@download/ENCFF981ISM.bed.gz to outputs/ENCFF981ISM-1_TssA/ENCFF981ISM.bed.gz
-> Verifying checksum
-> Downloaded file checksum: 9c193f16fb52a60daa00555fc986a6a1


## Obtaining chromatin state data

We will use the hg38 liftover of ROADMAP chromatin states that was already downloaded to [data/raw/roadmap](data/raw/roadmap). See README in this directory on how to obtain it.

In [14]:
ROADMAP_CHROMATIN_STATES_TGZ = RAW_DATA_DIR / 'roadmap/all_hg38lift.mnemonics.bedFiles.2021-11-25.tgz'
assert ROADMAP_CHROMATIN_STATES_TGZ.exists()

We need to extract the chromatin state information for the correct cell line from this archive.

We will assume cell line K562, whose ROADMAP Epigenome identifier is `E123`.
You can find identifiers for other cell lines [in the Roadmap metadata spreadsheet](https://egg2.wustl.edu/roadmap/web_portal/meta.html).

In [15]:
EPIGENOME_IDENTIFIER = 'E123' # K562
ROADMAP_CHROMATIN_STATE_FILENAME = f'{EPIGENOME_IDENTIFIER}_15_coreMarks_hg38lift_mnemonics.bed.gz'
CHROMATIN_STATE_FILE = NOTEBOOK_OUTPUT_DIR / ROADMAP_CHROMATIN_STATE_FILENAME

In [16]:
print(f"Extracting chromatin states for {EPIGENOME_IDENTIFIER} to file {CHROMATIN_STATE_FILE}")
print("Will run the following shell command:")
_command = ['tar', '-xOzf', ROADMAP_CHROMATIN_STATES_TGZ, ROADMAP_CHROMATIN_STATE_FILENAME]
print(' '.join(map(str,_command)))

import subprocess
with open(CHROMATIN_STATE_FILE, 'wb') as f:
    subprocess.run(_command, stdout=f, check=True)
    
print("Done !")
assert CHROMATIN_STATE_FILE.exists()

Extracting chromatin states for E123 to file outputs/ENCFF981ISM-1_TssA/E123_15_coreMarks_hg38lift_mnemonics.bed.gz
Will run the following shell command:
tar -xOzf ../../data/raw/roadmap/all_hg38lift.mnemonics.bedFiles.2021-11-25.tgz E123_15_coreMarks_hg38lift_mnemonics.bed.gz
Done !


The first ten rows of this file will give an indication on how the data is structured:

In [17]:
import gzip
with gzip.open(CHROMATIN_STATE_FILE, 'rt') as f:
    
    n_rows = 10
    for line in f:
        print(line.strip())
        n_rows -= 1
        if n_rows <= 0:
            break

chr1	10000	10800	9_Het
chr1	10800	96080	15_Quies
chr1	96276	96476	15_Quies
chr1	97276	109000	15_Quies
chr1	109000	109200	7_Enh
chr1	109200	118400	15_Quies
chr1	118400	118800	1_TssA
chr1	118800	138400	15_Quies
chr1	138400	139400	1_TssA
chr1	139400	177200	15_Quies


### Non-overlaping genomic bins

We start the data processing by dividing the whole genome into a set of non-overlapping bins of the specified bin size (parameter `BIN_SIZE`). This is achieved with [`bedtools makewindows`](https://daler.github.io/pybedtools/autodocs/pybedtools.bedtool.BedTool.window_maker.html).

In [18]:
import pybedtools

In [19]:
genomic_windows = pybedtools.BedTool().window_maker(w=BIN_SIZE, g=CHROMSIZES, i='srcwinnum')
genomic_windows = genomic_windows.saveas(NOTEBOOK_OUTPUT_DIR / 'genomic_windows.bed.gz')

In [20]:
print("Made {:,} genomic windows of bin size {:,}bp".format(len(genomic_windows), BIN_SIZE))

Made 3,031,053 genomic windows of bin size 1,000bp


Now throw away the genomic windows that overlap blacklisted regions

In [21]:
blacklist = pybedtools.BedTool(BLACKLIST)

In [22]:
genomic_windows_no_blacklist = genomic_windows.intersect(blacklist, v=True)
genomic_windows_no_blacklist.saveas(NOTEBOOK_OUTPUT_DIR / 'genomic_windows_wo_blacklist.bed.gz')


<BedTool(outputs/ENCFF981ISM-1_TssA/genomic_windows_wo_blacklist.bed.gz)>

In [23]:
print("Was left with {:,} genomic windows of bin size {:,}bp".format(len(genomic_windows_no_blacklist), BIN_SIZE))

Was left with 2,835,235 genomic windows of bin size 1,000bp


### genomic-bins vs bedfile intersection matrix

Once we have the genomic windows matrix we can use it as a common ground on which to project the ChIP-seq datasets onto. 

For each genomic bin, we want to obtain the maximum [`signalValue` (column 7)](https://genome.ucsc.edu/FAQ/FAQformat.html#format12) of the peaks from the `bed` files that overlap this bin. We can achieve this with [`bedtools map`](https://bedtools.readthedocs.io/en/latest/content/tools/map.html) operation.

In [24]:
matrix_to_bedtool_maps = {}

for id_, bed_filename in  BED_FILENAMES.items():
    print(f"-> Processing {bed_filename}")
    
    # Load bed filename
    bed = pybedtools.BedTool(bed_filename)
    # Drop chromosomes we are not interested in
    bed = bed.filter(lambda x: x.chrom in VALID_CHROMOSOMES)
    # Sort it
    bed = bed.sort(g=CHROMSIZES)
    
    # Map this file onto the genomic window
    mapped = genomic_windows_no_blacklist.map(
        bed,
        
        c=7, # signalValue
        o='max'
    )
    
    new_filename = (NOTEBOOK_OUTPUT_DIR / f'{id_}.mapped_to_genomic_windows.bed.gz')
    print(f'-> Done. Saving as {new_filename}')
    mapped.saveas(new_filename)
    
    matrix_to_bedtool_maps[id_] = mapped
    

-> Processing outputs/ENCFF981ISM-1_TssA/ENCFF981ISM.bed.gz
-> Done. Saving as outputs/ENCFF981ISM-1_TssA/ENCFF981ISM.mapped_to_genomic_windows.bed.gz


What the code above does result in is the bed file whose regions are the same of the genomic windows, but the fifth column (score) now corresponds to the maximum signal value overlapping peaks, or a dot (`.`) if the overlap is missing:

In [25]:
matrix_to_bedtool_maps[ENCODE_IDENTIFIER_BEDFILE_X].head(20)

chr1	793000	794000	chr1_794	.
 chr1	794000	795000	chr1_795	.
 chr1	795000	796000	chr1_796	.
 chr1	796000	797000	chr1_797	.
 chr1	797000	798000	chr1_798	.
 chr1	798000	799000	chr1_799	.
 chr1	799000	800000	chr1_800	.
 chr1	800000	801000	chr1_801	.
 chr1	801000	802000	chr1_802	.
 chr1	802000	803000	chr1_803	.
 chr1	803000	804000	chr1_804	.
 chr1	804000	805000	chr1_805	.
 chr1	805000	806000	chr1_806	.
 chr1	806000	807000	chr1_807	.
 chr1	807000	808000	chr1_808	.
 chr1	808000	809000	chr1_809	.
 chr1	809000	810000	chr1_810	.
 chr1	810000	811000	chr1_811	.
 chr1	811000	812000	chr1_812	.
 chr1	812000	813000	chr1_813	.
 

### Chromatin states vs bedfile intersection matrix

We have to process the chromatin state bed file slightly differently.
For each genomic bin, we want to obtain _all_ of the chromatin states that overlap this bin.

To do this, we need to aggregate over column 4 (name/mnemonic), and use bedtools operation "distinct" (see [bedtools documentation](https://bedtools.readthedocs.io/en/latest/content/tools/map.html)).

In [26]:
for id_, bed_filename in [('chromatin_states', CHROMATIN_STATE_FILE)]:
    print(f"-> Processing {bed_filename}")
    
    # Load bed filename
    bed = pybedtools.BedTool(bed_filename)
    
    # Drop chromosomes we are not interested in
    bed = bed.filter(lambda x: x.chrom in VALID_CHROMOSOMES)
    
    # Sort it
    bed = bed.sort(g=CHROMSIZES)
    
    # Map this file onto the genomic window
    mapped = genomic_windows_no_blacklist.map(
        bed,
        
        c=4, # Mnemonic
        o='distinct'
    )
    
    new_filename = (NOTEBOOK_OUTPUT_DIR / f'{id_}.mapped_to_genomic_windows.bed.gz')
    print(f'-> Done. Saving as {new_filename}')
    mapped.saveas(new_filename)
    
    matrix_to_bedtool_maps[id_] = mapped
    

-> Processing outputs/ENCFF981ISM-1_TssA/E123_15_coreMarks_hg38lift_mnemonics.bed.gz
-> Done. Saving as outputs/ENCFF981ISM-1_TssA/chromatin_states.mapped_to_genomic_windows.bed.gz


The resulting bed file looks like this. Note how bins overlapping multiple chromatin states
have them all listed, separated with commas:

In [27]:
matrix_to_bedtool_maps['chromatin_states'].head(20)

chr1	793000	794000	chr1_794	15_Quies
 chr1	794000	795000	chr1_795	15_Quies
 chr1	795000	796000	chr1_796	15_Quies
 chr1	796000	797000	chr1_797	15_Quies
 chr1	797000	798000	chr1_798	15_Quies
 chr1	798000	799000	chr1_799	15_Quies
 chr1	799000	800000	chr1_800	15_Quies
 chr1	800000	801000	chr1_801	15_Quies
 chr1	801000	802000	chr1_802	15_Quies
 chr1	802000	803000	chr1_803	15_Quies
 chr1	803000	804000	chr1_804	15_Quies
 chr1	804000	805000	chr1_805	15_Quies,7_Enh
 chr1	805000	806000	chr1_806	5_TxWk,7_Enh
 chr1	806000	807000	chr1_807	5_TxWk
 chr1	807000	808000	chr1_808	5_TxWk
 chr1	808000	809000	chr1_809	5_TxWk
 chr1	809000	810000	chr1_810	5_TxWk
 chr1	810000	811000	chr1_811	5_TxWk
 chr1	811000	812000	chr1_812	5_TxWk
 chr1	812000	813000	chr1_813	1_TssA,2_TssAFlnk,5_TxWk
 

At this point we can get out of the bedtools environment and work in pandas:

In [28]:
matrix_to_bedtool_maps_df = {
    id_: mapped_bed.to_dataframe()
    for id_, mapped_bed in matrix_to_bedtool_maps.items()
}

In [29]:
matrix_to_bedtool_maps_df[ENCODE_IDENTIFIER_BEDFILE_X]

,chrom,start,end,name,score
0,chr1,793000,794000,chr1_794,.
1,chr1,794000,795000,chr1_795,.
2,chr1,795000,796000,chr1_796,.
3,chr1,796000,797000,chr1_797,.
4,chr1,797000,798000,chr1_798,.
...,...,...,...,...,...
2835230,chrX,155978000,155979000,chrX_155979,.
2835231,chrX,155979000,155980000,chrX_155980,.
2835232,chrX,155980000,155981000,chrX_155981,.
2835233,chrX,155981000,155982000,chrX_155982,.


In [30]:
matrix_to_bedtool_maps_df['chromatin_states']

,chrom,start,end,name,score
0,chr1,793000,794000,chr1_794,15_Quies
1,chr1,794000,795000,chr1_795,15_Quies
2,chr1,795000,796000,chr1_796,15_Quies
3,chr1,796000,797000,chr1_797,15_Quies
4,chr1,797000,798000,chr1_798,15_Quies
...,...,...,...,...,...
2835230,chrX,155978000,155979000,chrX_155979,15_Quies
2835231,chrX,155979000,155980000,chrX_155980,15_Quies
2835232,chrX,155980000,155981000,chrX_155981,15_Quies
2835233,chrX,155981000,155982000,chrX_155982,15_Quies


### Converting matrices into a workable format: bed file

We are mostly interested in the `score` column here.
For the bed file X, the process is fairly straightforward:

1. Settthe index to the column `name`
2. Select only the `score` column
3. Replace `.` scores with NaNs
4. Convert to `float`

In [31]:
bed_matrix = pd.DataFrame(
    {
        id_: matrix_to_bedtool_maps_df[id_].set_index('name')['score'].replace('.', np.nan).astype(float)
        for id_ in [ENCODE_IDENTIFIER_BEDFILE_X]
    }
)
bed_matrix

,ENCFF981ISM
name,
chr1_794,NaN
chr1_795,NaN
chr1_796,NaN
chr1_797,NaN
chr1_798,NaN
...,...
chrX_155979,NaN
chrX_155980,NaN
chrX_155981,NaN


Since we cannot compute correlations between chromatin states (for that, see other examples), we will only need the boolean matrix:

In [32]:
bed_matrix_binary = ~bed_matrix.isnull()
bed_matrix_binary.head()

,ENCFF981ISM
name,
chr1_794,False
chr1_795,False
chr1_796,False
chr1_797,False
chr1_798,False


### Converting matrices into workable format: chromatin states

For chromatin states, however, the process is a bit more involved as we need to do the following:
    
1. Set the index to the column `name`
2. Select only the `score` column
3. Replace `.` scores with empty strings
4. Split the chromatin states by commas
5. Pick only the regions that contain the chromatin state of interest
6. Convert to `bool`

This complicated set of `pandas` commands will do the trick:

In [33]:
# Reindex and pick score
chromatin_state_matrix = matrix_to_bedtool_maps_df['chromatin_states'].set_index('name')['score'].copy()
# Replace dots with empty
chromatin_state_matrix = chromatin_state_matrix.replace('.', '').fillna('')
# Split states by ',', the expand=True followed with stack will generate results in the long form, see output:
chromatin_state_matrix = chromatin_state_matrix.astype(str).str.split(',', expand=True).stack()
# We want to be able to name the resulting columns
chromatin_state_matrix = chromatin_state_matrix.reset_index()
# Name is the same as bedfile name, "i" is an iterator which tracks which position the chromatin state was in
# and chromatin state is chromatin state
chromatin_state_matrix.columns = ['name', 'i', 'chromatin_state']
# Intermediate output to see what is happening better
chromatin_state_matrix.head(20)

,name,i,chromatin_state
0,chr1_794,0,15_Quies
1,chr1_795,0,15_Quies
2,chr1_796,0,15_Quies
3,chr1_797,0,15_Quies
4,chr1_798,0,15_Quies
5,chr1_799,0,15_Quies
6,chr1_800,0,15_Quies
7,chr1_801,0,15_Quies
8,chr1_802,0,15_Quies
9,chr1_803,0,15_Quies


In [34]:
# Pick the chromatin state of interest
chromatin_state_matrix = chromatin_state_matrix[chromatin_state_matrix['chromatin_state'] == CHROMATIN_STATE]
# index by name again
chromatin_state_matrix = chromatin_state_matrix.set_index('name')
# This should not be duplicated anymore
assert not chromatin_state_matrix.index.duplicated().any()

# Start converting to boolean by adding a column named as the state itself
chromatin_state_matrix[CHROMATIN_STATE] = True # (as we filtered the dataframe to contain only regions that contain state, this should be True)
# Keep only this column
chromatin_state_matrix = chromatin_state_matrix[CHROMATIN_STATE]
# Reindex matrix with the full list of regions, filling in with False 
chromatin_state_matrix = chromatin_state_matrix.reindex(genomic_windows_no_blacklist.to_dataframe()['name'].values, fill_value=False)

# We should have the matrix ready now:
chromatin_state_matrix

name
chr1_794       False
chr1_795       False
chr1_796       False
chr1_797       False
chr1_798       False
               ...  
chrX_155979    False
chrX_155980    False
chrX_155981    False
chrX_155982    False
chrX_155983    False
Name: 1_TssA, Length: 2835235, dtype: bool

We can make sure this is boolean (not object):

In [35]:
chromatin_state_matrix.dtype

dtype('bool')

And see that it has both True and False

In [36]:
chromatin_state_matrix.value_counts()

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


False    2793552
True       41683
Name: 1_TssA, dtype: int64

At this point we can simply join our matrix with the binary bed file matrix:

In [37]:
bed_matrix_binary = bed_matrix_binary.join(chromatin_state_matrix)
bed_matrix_binary

,ENCFF981ISM,1_TssA
name,,
chr1_794,False,False
chr1_795,False,False
chr1_796,False,False
chr1_797,False,False
chr1_798,False,False
...,...,...
chrX_155979,False,False
chrX_155980,False,False
chrX_155981,False,False


There should not be any nulls in this matrix:

In [38]:
assert not bed_matrix_binary.isnull().any().any()

And dtypes should be bool:

In [39]:
bed_matrix_binary.dtypes

ENCFF981ISM    bool
1_TssA         bool
dtype: object

### Entropy based statistics

The entropy-based statistics are the main workhorse of this pipeline.

To make use of it, we need to compute the 2x2 co-occurence matrix for the two datasets

In [40]:
import itertools

cooccurrence_matrix = []

# For all four combinations of [False, True] 
for x_value, y_value in itertools.product([False, True], repeat=2):
    
    # Count how many times they occurr
    mask = bed_matrix_binary[ENCODE_IDENTIFIER_BEDFILE_X] == x_value
    mask &= bed_matrix_binary[CHROMATIN_STATE] == y_value
    count = mask.sum()
    
    # Store that
    cooccurrence_matrix.append([x_value, y_value, count])

# Make this into a dataframe
cooccurrence_matrix = pd.DataFrame(
    cooccurrence_matrix, 
    columns=[
         ENCODE_IDENTIFIER_BEDFILE_X,
         CHROMATIN_STATE,
         'count',
     ]
)

# Reshape this into 2x2 matrix
cooccurrence_matrix = cooccurrence_matrix.set_index([
    ENCODE_IDENTIFIER_BEDFILE_X, 
    CHROMATIN_STATE
])['count'].unstack(CHROMATIN_STATE)

From this we get the number of times the datasets overlap (based on our genomic grid), the number of times they are both missing, and the number of times one occurs without the other (see below).

Note that in the code we use some optimisations and therefore the numbers are calculated in a slightly more sophisticated way, but the end result is the same.

In [41]:
cooccurrence_matrix

1_TssA,False,True
ENCFF981ISM,,
False,2785700,19012
True,7852,22671


Note that the matrix columns sum to the universe length (i.e. the number of genomic windows):

In [42]:
universe_length = len(genomic_windows_no_blacklist)
_sum = np.sum(np.asarray(cooccurrence_matrix))
print("Universe length: {:,}, sum of co-occurrence matrix columns: {:,}".format(universe_length, _sum))
assert universe_length == _sum


Universe length: 2,835,235, sum of co-occurrence matrix columns: 2,835,235


Once we have the co-occurrence matrix, we can compute the joint probabilities.

To add some statistical smoothing we will use a pseudocount defined in `STATISTICS_PSEUDOCOUNT` above:

In [43]:
print(f'{STATISTICS_PSEUDOCOUNT=}')

STATISTICS_PSEUDOCOUNT=100


We will add this pseudocount to each and every cell of the co-occurrence matrix, and will then divide the cells by the total effective universe size which is now equal to the `universe_size + 4*pseudocount`.

In [44]:
joint_probabilities = (cooccurrence_matrix + STATISTICS_PSEUDOCOUNT) / (universe_length + 4*STATISTICS_PSEUDOCOUNT)

In [45]:
joint_probabilities

1_TssA,False,True
ENCFF981ISM,,
False,0.982425,0.00674
True,0.002804,0.00803


We can obtain marginal probabilities as well:

In [46]:
marginal_x = joint_probabilities.sum(axis=1)
assert marginal_x.index.name == ENCODE_IDENTIFIER_BEDFILE_X

marginal_y = joint_probabilities.sum(axis=0)
assert marginal_y.index.name == CHROMATIN_STATE


In [47]:
marginal_x

ENCFF981ISM
False    0.989165
True     0.010835
dtype: float64

In [48]:
marginal_y

1_TssA
False    0.98523
True     0.01477
dtype: float64

If our data were independent, we would expect to observe the following joint_probability table (resulting of multiplying the marginal probabilities):

In [49]:
import itertools

joint_independent = []

# For all four combinations of [False, True] 
for x_value, y_value in itertools.product([False, True], repeat=2):
    
    # Compute the probability under independence
    p = marginal_x.loc[x_value] * marginal_y.loc[y_value]
    
    # Store that
    joint_independent.append([x_value, y_value, p])

# Make this into a dataframe
joint_independent = pd.DataFrame(
    joint_independent, 
    columns=[
         ENCODE_IDENTIFIER_BEDFILE_X,
         CHROMATIN_STATE,
         'count',
     ]
)

# Reshape this into 2x2 matrix
joint_independent = joint_independent.set_index([
    ENCODE_IDENTIFIER_BEDFILE_X, 
    CHROMATIN_STATE
])['count'].unstack(CHROMATIN_STATE)

In [50]:
assert joint_independent.index.equals(joint_probabilities.index)
assert joint_independent.columns.equals(joint_probabilities.columns)


In [51]:
joint_independent

1_TssA,False,True
ENCFF981ISM,,
False,0.974555,0.01461
True,0.010675,0.00016


We now have everything we need to compute the marginal and relative entropies.
First, the marginal entropies:

In [52]:
import scipy.stats 

entropy_marginal_x = scipy.stats.entropy(marginal_x) 
print(f'-> Computed marginal entropy of {ENCODE_IDENTIFIER_BEDFILE_X} = {entropy_marginal_x}')

entropy_marginal_y = scipy.stats.entropy(marginal_y) 
print(f'-> Computed marginal entropy of {CHROMATIN_STATE} = {entropy_marginal_y}')


-> Computed marginal entropy of ENCFF981ISM = 0.05980241823702094
-> Computed marginal entropy of 1_TssA = 0.07691924810266863


Now we can compute the joint entropy of the two variables:

In [53]:
joint_entropy_xy = scipy.stats.entropy(np.asarray(joint_probabilities).ravel())
print(f'-> Computed joint entropy of {ENCODE_IDENTIFIER_BEDFILE_X},{CHROMATIN_STATE} = {joint_entropy_xy}')


-> Computed joint entropy of ENCFF981ISM,1_TssA = 0.10633914807473072


We can also compute the mutual information, which is the K-L divergence from independence:

In [54]:
mi_xy_using_kl_formula = scipy.stats.entropy(
    np.asarray(joint_probabilities).ravel(),
    np.asarray(joint_independent).ravel(),
    
)
print(f'-> Computed MI of {ENCODE_IDENTIFIER_BEDFILE_X},{CHROMATIN_STATE} = {mi_xy_using_kl_formula}')


-> Computed MI of ENCFF981ISM,1_TssA = 0.03038251826495885


This should be the same as the sum of marginal entropies minus joint entropy:

In [55]:
mi_xy_using_entropies = entropy_marginal_x + entropy_marginal_y - joint_entropy_xy
print(f'-> Recomputed MI of {ENCODE_IDENTIFIER_BEDFILE_X},{CHROMATIN_STATE} = {mi_xy_using_entropies}')

from numpy.testing import assert_array_almost_equal
# I'm using the array function because the default parameters make sense there
assert_array_almost_equal([mi_xy_using_kl_formula], [mi_xy_using_entropies])

-> Recomputed MI of ENCFF981ISM,1_TssA = 0.030382518264958858


The resulting uncertainty coefficients are therefore computed as follows:

In [56]:
uncertainty_x_given_y = mi_xy_using_kl_formula / entropy_marginal_x
uncertainty_y_given_x = mi_xy_using_kl_formula / entropy_marginal_y

In [57]:
print(f"-> Uncertainty coefficient U(X|Y) <- MI(X,Y)/H(X) =  {uncertainty_x_given_y}")
print(f"-> Uncertainty coefficient U(Y|X) <- MI(X,Y)/H(Y) =  {uncertainty_y_given_x}")


-> Uncertainty coefficient U(X|Y) <- MI(X,Y)/H(X) =  0.50804832246985
-> Uncertainty coefficient U(Y|X) <- MI(X,Y)/H(Y) =  0.39499239805887754


Note that in the pipeline output we will be using the convention that  𝑋  is in rows of the output, and  𝑌  in the columns. And we will further assume that  𝑋  is a protein, e.g. PHF8 and Y is a chromatin state. With such assumptions we will be using the uncertainty_x_given_y as our final output. The code below verifies this and shows how to extract these numbers from the pipeline

In [58]:
uncertainty_x_given_y

0.50804832246985

# Verification of pipeline results

As we used two encode datasets to comptue this example we can use this notebook to verify the pipeline results, provided that it has been run.

In [59]:
cell_line = encode_meta['Biosample term name'].unique()
assert len(cell_line) == 1
cell_line = cell_line[0]

In [60]:
OUTPUT_CSV_GZ = pathlib.Path(
    f'../../output/final/analysis/params_{BIN_SIZE}bp_pc_{STATISTICS_PSEUDOCOUNT}_mp_{MIN_PERIODS_FOR_CORRELATIONS}/{cell_line}/consolidated_tables/bedstats_consolidated_{cell_line}_{BIN_SIZE}bp_params_pc_{STATISTICS_PSEUDOCOUNT}_mp_{MIN_PERIODS_FOR_CORRELATIONS}_from_bed.csv.gz'
)

print(f'-> Reading pipeline outputs from {OUTPUT_CSV_GZ}')
assert OUTPUT_CSV_GZ.is_file(), "Pipeline output not found - are you sure the pipeline was run to completion?"


pipeline_outputs = pd.read_csv(OUTPUT_CSV_GZ, index_col=0)

pipeline_outputs.columns = pd.MultiIndex.from_tuples([c.split('__') for c in pipeline_outputs.columns], names=['col_type', 'colname'])

print('-> column headers parsed:')
print(pipeline_outputs.columns.get_level_values('col_type').unique())


-> Reading pipeline outputs from ../../output/final/analysis/params_1000bp_pc_100_mp_1/K562/consolidated_tables/bedstats_consolidated_K562_1000bp_params_pc_100_mp_1_from_bed.csv.gz
-> column headers parsed:
Index(['metadata', 'marcs_feature_significant_category', 'normalised_mi',
       'entropy_by_row', 'kendall_correlation', 'mi', 'entropy_by_col',
       'counts_true_true', 'counts_false_true', 'counts_true_false',
       'counts_false_false'],
      dtype='object', name='col_type')


In [61]:
pipeline_outputs.head()

col_type                    metadata                                \
colname                    encode_id cell_line  factor factor_type   
Factor_Cell_Identifier                                               
ADNP-K562-ENCFF739AJO    ENCFF739AJO      K562    ADNP     protein   
AFF1-K562-ENCFF195YGC    ENCFF195YGC      K562    AFF1     protein   
AFF1-K562-ENCFF674XTY    ENCFF674XTY      K562    AFF1     protein   
ARID1B-K562-ENCFF879NTL  ENCFF879NTL      K562  ARID1B     protein   
ARID2-K562-ENCFF913WRW   ENCFF913WRW      K562   ARID2     protein   

col_type                                 marcs_feature_significant_category  \
colname                 marcs_gene_label                              H2A.Z   
Factor_Cell_Identifier                                                        
ADNP-K562-ENCFF739AJO               ADNP                            Neither   
AFF1-K562-ENCFF195YGC               AFF1                                NaN   
AFF1-K562-ENCFF674XTY               AFF1                                NaN   
ARID1B-K562-ENCFF879NTL           ARID1B                            Neither   
ARID2-K562-ENCFF913WRW             ARID2                            Neither   

col_type                                                    ...  \
colname                 H3K27ac H3K27me2 H3K27me3  H3K4me1  ...   
Factor_Cell_Identifier                                      ...   
ADNP-K562-ENCFF739AJO       NaN      NaN  Neither  Neither  ...   
AFF1-K562-ENCFF195YGC       NaN      NaN      NaN      NaN  ...   
AFF1-K562-ENCFF674XTY       NaN      NaN      NaN      NaN  ...   
ARID1B-K562-ENCFF879NTL     NaN      NaN      NaN      NaN  ...   
ARID2-K562-ENCFF913WRW      NaN      NaN  Neither  Neither  ...   

col_type                counts_false_false                                \
colname                     state:15_Quies state:1_TssA state:2_TssAFlnk   
Factor_Cell_Identifier                                                     
ADNP-K562-ENCFF739AJO             944095.0    2770050.0        2758028.0   
AFF1-K562-ENCFF195YGC             949267.0    2779125.0        2765182.0   
AFF1-K562-ENCFF674XTY             957511.0    2788113.0        2772685.0   
ARID1B-K562-ENCFF879NTL           912195.0    2738438.0        2735645.0   
ARID2-K562-ENCFF913WRW            958075.0    2784414.0        2769572.0   

col_type                                                                     \
colname                 state:3_TxFlnk state:4_Tx state:5_TxWk state:6_EnhG   
Factor_Cell_Identifier                                                        
ADNP-K562-ENCFF739AJO        2796910.0  2701476.0    2416319.0    2777986.0   
AFF1-K562-ENCFF195YGC        2803088.0  2706542.0    2417014.0    2782919.0   
AFF1-K562-ENCFF674XTY        2814105.0  2717880.0    2426518.0    2793931.0   
ARID1B-K562-ENCFF879NTL      2760289.0  2663106.0    2385776.0    2742981.0   
ARID2-K562-ENCFF913WRW       2812570.0  2717226.0    2427008.0    2792948.0   

col_type                                                          
colname                 state:7_Enh state:8_ZNF/Rpts state:9_Het  
Factor_Cell_Identifier                                            
ADNP-K562-ENCFF739AJO     2652595.0        2800345.0   2776332.0  
AFF1-K562-ENCFF195YGC     2653527.0        2805917.0   2781939.0  
AFF1-K562-ENCFF674XTY     2660970.0        2817366.0   2793315.0  
ARID1B-K562-ENCFF879NTL   2642590.0        2761588.0   2737625.0  
ARID2-K562-ENCFF913WRW    2660247.0        2816181.0   2792197.0  

[5 rows x 326 columns]

Pipeline outputs are indexed using the Factor-cell-identifier keys, create those here:

In [62]:
def factor_cell_identifier(encode_meta_row):
    assay = encode_meta_row['Assay']

    if assay in ['TF ChIP-seq', 'Histone ChIP-seq']:
        factor = encode_meta_row['Experiment target'].partition('-')[0]
    else:
        factor = assay
        
    cell_line = encode_meta_row['Biosample term name']
    identifier = encode_meta_row.name
    
    return f'{factor}-{cell_line}-{identifier}'

In [63]:
FACTOR_CELL_IDENTIFIER_X = factor_cell_identifier(encode_meta.loc[ENCODE_IDENTIFIER_BEDFILE_X])
print(f'-> {ENCODE_IDENTIFIER_BEDFILE_X} -> {FACTOR_CELL_IDENTIFIER_X}')

-> ENCFF981ISM -> PHF8-K562-ENCFF981ISM


The chromatin states in the outputs have a prefix `state:`

In [64]:
RESULTS_ID_CHROMATIN_STATE = 'state:' + CHROMATIN_STATE

### Validation of the co-occurrence counts

Co-occurrence counts are split between four columns in the pipeline output:

1. `counts_true_true`, representing X=True, Y=True (X in rows)
2. `counts_false_true`, representing X=False, Y=True (X in rows)
3. `counts_true_false`, representing X=True, Y=False (X in rows)
4. `counts_false_false` representing X=False, Y=False (X in rows)


In [65]:
FACTOR_CELL_IDENTIFIER_X, RESULTS_ID_CHROMATIN_STATE

('PHF8-K562-ENCFF981ISM', 'state:1_TssA')

In [66]:
cooccurrence_matrix

1_TssA,False,True
ENCFF981ISM,,
False,2785700,19012
True,7852,22671


In [67]:
co_occurrence_matrix_from_pipeline = pd.DataFrame([
    [
     pipeline_outputs.loc[FACTOR_CELL_IDENTIFIER_X, ('counts_false_false', RESULTS_ID_CHROMATIN_STATE)],
     pipeline_outputs.loc[FACTOR_CELL_IDENTIFIER_X, ('counts_false_true', RESULTS_ID_CHROMATIN_STATE)],
    ],
    [
     pipeline_outputs.loc[FACTOR_CELL_IDENTIFIER_X, ('counts_true_false', RESULTS_ID_CHROMATIN_STATE)],
     pipeline_outputs.loc[FACTOR_CELL_IDENTIFIER_X, ('counts_true_true', RESULTS_ID_CHROMATIN_STATE)],
    ],
], index=[False, True], columns=[False, True], dtype=int)

co_occurrence_matrix_from_pipeline.index.name = ENCODE_IDENTIFIER_BEDFILE_X
co_occurrence_matrix_from_pipeline.columns.name = RESULTS_ID_CHROMATIN_STATE

co_occurrence_matrix_from_pipeline

state:1_TssA,False,True
ENCFF981ISM,,
False,2785700,19012
True,7852,22671


Make sure the results are identical:

In [68]:
from numpy.testing import assert_array_equal
assert_array_equal(cooccurrence_matrix, co_occurrence_matrix_from_pipeline)

### Validation of the entropies

We can now validate the marginal entropies.

For the $X$ variable, these are listed in the pipeline matrix only once in column `entropy_by_row`:

In [69]:
marginal_entropy_x_from_pipeline = pipeline_outputs.loc[
    FACTOR_CELL_IDENTIFIER_X, 
    ('entropy_by_row', 'entropy_by_row')
]

print(f'-> Marginal entropy for {ENCODE_IDENTIFIER_BEDFILE_X}: computed here: {entropy_marginal_x}, computed by pipeline: {marginal_entropy_x_from_pipeline}')
assert_array_almost_equal([entropy_marginal_x], [marginal_entropy_x_from_pipeline])

-> Marginal entropy for ENCFF981ISM: computed here: 0.05980241823702094, computed by pipeline: 0.0598024182370209


For chromatin state, the entropy is listed as one of the columns in `entropy_by_col`.

In [70]:
marginal_entropy_y_from_pipeline_by_col = pipeline_outputs[('entropy_by_col', RESULTS_ID_CHROMATIN_STATE)].unique()

print(f'-> Marginal entropy for {CHROMATIN_STATE}: computed here: {entropy_marginal_y}, computed by pipeline (column): {marginal_entropy_y_from_pipeline_by_col}')


assert_array_almost_equal([entropy_marginal_y], marginal_entropy_y_from_pipeline_by_col)



-> Marginal entropy for 1_TssA: computed here: 0.07691924810266863, computed by pipeline (column): [0.07691925]


Similarly, we can verify Mutual Information calculations:

In [71]:
mi_from_pipeline = pipeline_outputs.loc[FACTOR_CELL_IDENTIFIER_X, ('mi', RESULTS_ID_CHROMATIN_STATE)]

print(f'-> MI for {ENCODE_IDENTIFIER_BEDFILE_X}, {CHROMATIN_STATE}: computed here: {mi_xy_using_kl_formula}, computed by pipeline: {mi_from_pipeline}')

assert_array_almost_equal([mi_xy_using_kl_formula], [mi_from_pipeline])

-> MI for ENCFF981ISM, 1_TssA: computed here: 0.03038251826495885, computed by pipeline: 0.0303825182649588


And finally, and most importantly we need to verify that the normalised mutual information column in the pipeline output correspond to the `uncertainty_x_given_y`, and _not_ `uncertainty_y_given_x`

In [72]:
normed_mi_from_pipeline = pipeline_outputs.loc[FACTOR_CELL_IDENTIFIER_X, ('normalised_mi', RESULTS_ID_CHROMATIN_STATE)]

print(f'-> Normed MI {ENCODE_IDENTIFIER_BEDFILE_X} "given" {CHROMATIN_STATE}: computed here: {uncertainty_x_given_y}, computed by pipeline: {normed_mi_from_pipeline}')

assert_array_almost_equal([uncertainty_x_given_y], [normed_mi_from_pipeline])

print(f'-> Note this is not the same as normed MI {CHROMATIN_STATE} "given" {ENCODE_IDENTIFIER_BEDFILE_X} (={uncertainty_y_given_x})')


-> Normed MI ENCFF981ISM "given" 1_TssA: computed here: 0.50804832246985, computed by pipeline: 0.5080483224698498
-> Note this is not the same as normed MI 1_TssA "given" ENCFF981ISM (=0.39499239805887754)


That's it!